In [16]:
import pandas as pd
import numpy as np
import matplotlib as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)
%matplotlib inline

In [11]:
df= pd.read_csv('../01.Data_Collection/screen_show_seat_updated.csv')

In [8]:
df.shape

(70845, 19)

헷갈리지 않기 위한 용어 정리
- 좌석판매율 계산법: 관객수(실제 티켓구매한 관객)/좌석수(스케줄에 잡힌 좌석수)
e.g. (751754/964153)*100
- 스크린점유율: 해당 집계일의 영화스크린수/총 스크린수
- 좌석점유율: 해당 집계일의 영화좌석수/총 좌석수
- 상영점유율: 해당 집계일의 영화상영횟수/총 상영횟수
총 갯수는 해외영화 포함이므로 해당 점유율은 정확하지 않고 직접 계산해야 할 수 있음

In [100]:
df.head()

,스크린점유율순위,영화명,개봉일,스크린수,스크린점유율,매출액,누적매출액,관객수,누적관객수,점유율집계일,좌석판매점유율순위,좌석판매율,좌석점유율,좌석수,순위,상영횟수,상영점유율
0,1,국제시장,2014-12-17,941,35.8%,6129982500,47219550096,751754,6097431,2015-01-01,1,78.0%,34.99%,964153,1,4650,45.651%
1,2,"님아, 그 강을 건너지 마오",2014-11-27,473,18.0%,1479015600,31324845079,178941,4025319,2015-01-01,3,67.2%,9.66%,266197,2,1979,19.429%
2,3,기술자들,2014-12-24,464,17.7%,1299883700,14952363848,160371,1938454,2015-01-01,2,57.9%,10.05%,276896,3,1886,18.516%
3,4,상의원,2014-12-24,258,9.8%,230484800,5524209364,28439,705071,2015-01-01,5,46.6%,2.22%,61095,5,491,4.82%
4,5,개를 훔치는 완벽한 방법,2014-12-31,196,7.5%,299346400,608002300,37741,80633,2015-01-01,4,48.1%,2.85%,78410,4,650,6.381%


In [101]:
#좌석판매율 계산법: 관객수(실제 티켓구매한 관객)/좌석수(스케줄에 잡힌 좌석수)
(751754/964153)*100
# 스크린점유율: 해당 집계일의 영화스크린수/총 스크린수
# 좌석점유율: 해당 집계일의 영화좌석수/총 좌석수

77.97040511205172

In [18]:
df.isnull().sum()

스크린점유율순위        0
영화명             0
개봉일          5515
스크린수            0
스크린점유율          0
매출액             0
누적매출액           0
관객수             0
누적관객수           0
점유율집계일          0
좌석판매점유율순위       0
좌석판매율           0
좌석점유율           0
좌석수             0
순위              0
상영횟수            0
상영점유율           0
dtype: int64

In [31]:
print(df[df['개봉일'].isnull()]['영화명'].nunique(), '건의 영화가 개봉일이 없음')
print(df['점유율집계일'].nunique(),'일의 수집')

1781 건의 영화가 개봉일이 없음
1771 일의 수집


In [25]:
pd.DataFrame(df[df['개봉일'].isnull()]['영화명'].unique())

,0
0,뽀로로 극장판 눈요정 마을 대모험
1,민우씨 오는 날
2,2015 우수 단편영화 상영 01
3,고종황제와 의사 안중근
4,1월 단편 상상극장 – 맛있게 먹겠습니다
5,맨발의 청춘
6,떠날 때는 말없이
7,2015 우수 단편영화 상영 02
8,독립영화발표회(251회) - 박영석 프로그래머 추천전
9,도도하라


개봉일 없는 영화들: 거의 영화제 작품, 단편, 인디작품 등

In [36]:
print(df.groupby('점유율집계일').count().min())
print(df.groupby('점유율집계일').count().max())
#하루당 최저 13개에서 76개까지 영화 있음

스크린점유율순위     13
영화명          13
개봉일          12
스크린수         13
스크린점유율       13
매출액          13
누적매출액        13
관객수          13
누적관객수        13
좌석판매점유율순위    13
좌석판매율        13
좌석점유율        13
좌석수          13
순위           13
상영횟수         13
상영점유율        13
dtype: int64
스크린점유율순위     76
영화명          76
개봉일          66
스크린수         76
스크린점유율       76
매출액          76
누적매출액        76
관객수          76
누적관객수        76
좌석판매점유율순위    76
좌석판매율        76
좌석점유율        76
좌석수          76
순위           76
상영횟수         76
상영점유율        76
dtype: int64


In [113]:
left =pd.DataFrame(df.groupby(['점유율집계일'])['매출액'].max()).reset_index()

In [114]:
left

,점유율집계일,매출액
0,2015-01-01,6129982500
1,2015-01-02,3782877729
2,2015-01-03,5102642636
3,2015-01-04,4437089200
4,2015-01-05,1558134100
5,2015-01-06,1550265000
6,2015-01-07,1398815300
7,2015-01-08,1413971800
8,2015-01-09,1663760900
9,2015-01-10,3746238400


In [64]:
right =df[['매출액','영화명']]

In [144]:
top_movie_per_day =left.merge(right, on='매출액', how='inner')

In [143]:
top_movie_per_day.head()

,점유율집계일,매출액,영화명
0,2015-01-01,6129982500,국제시장
1,2015-01-02,3782877729,국제시장
2,2015-01-03,5102642636,국제시장
3,2015-01-04,4437089200,국제시장
4,2015-01-05,1558134100,국제시장


In [133]:
print(top_movie_per_day['영화명'].nunique(),'개의 영화가 1등을 함')

148 개의 영화가 1등을 함


매출액 1등한 영화들과 그 횟수

기생충	43회

In [140]:
top_movie_per_day.groupby('영화명').count()

,점유율집계일,매출액
영화명,,
1987,9,9
7호실,7,7
82년생 김지영,15,15
PMC: 더 벙커,14,14
가려진 시간,7,7
가장 보통의 연애,21,21
간신,14,14
강남 1970,14,14
강철비,6,6


그외에도 1등한 영화들의 좌석점유율을 보자

가설
- 좌석수가 높은데 좌석 판매율이 낮은 영화는 무엇인가?
- 좌석수가 낮은데 좌석 판매율이 높은 영화는 무엇인가?
- 좌석수와 좌석 판매율의 상관관계